In [2]:
import re
import json
import pytz
import time
import requests
import random
from fake_useragent import UserAgent
from celery import Celery
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import rethinkdb as r
from pymongo import MongoClient
from bson.objectid import ObjectId

In [3]:
def random_header(url):
    ua = UserAgent()
    random_header = json.loads(r'''{
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Host": "www.dogforum.com",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent":"%s"
    }'''%ua.random)
    random_header["Referer"] = url
    return random_header

In [ ]:
title_url = 'http://www.dogforum.com/dog-health/ladybugs-339090/'
page = 2
title = 'test'
resp = requests.get(title_url, headers=random_header(title_url))
resp.encoding = 'cp1252'
soup = BeautifulSoup(resp.text, 'lxml')

In [ ]:
soup.select('meta')

In [ ]:
soup.select('.tborder.vbseo_like_postbit')

In [ ]:
re.findall('http://www.dogforum.com/(.*)',title_url)[0]

In [ ]:
len(soup.select('.tborder.vbseo_like_postbit'))

# Deal with quotation

In [ ]:
[x.extract() for x in soup.select_one('.jobname_title').select('span')]

In [ ]:
# method 1
quotation = []
quote = soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')
for qno in range(0,len(quote)):
    quotation.append(quote[qno].text.strip())
quotation

In [ ]:
# method 2
quotation = [y.text.strip() for y in soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')]
quotation

# Method 2: quicky way to deal with content

In [ ]:
# extract quotations
[quo.extract() for quo in soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')]
soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))[0].text.strip().replace('Quote: ','[...quotation...]')

### Something dirty in tag that can't be removed via extract quotation only. Must do extraction of div then.

In [ ]:
[quo.extract() for quo in soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')]
div = soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))[0]
[a.extract() for a in div.select('div')]
soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))[0].text

In [ ]:
[quo.extract() for quo in soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')]
soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0].text.strip().replace('Quote: ','[...quotation...]')

###  However, [...quotation...] will be eliminated after extract div... sigh

In [ ]:
[quo.extract() for quo in soup.select('.tborder.vbseo_like_postbit')[0].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')]
div = soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0]
[a.extract() for a in div.select('div')]
soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0].text

# practice method 1 (but not necessary)

In [ ]:
title_url = 'http://www.dogforum.com/dog-health/what-parvo-5256/page2/'
resp = requests.get(title_url, headers=random_header(title_url))
resp.encoding = 'cp1252'
soup = BeautifulSoup(resp.text, 'lxml')

In [ ]:
quote = soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')
quotation = []
for qno in range(0,len(quote)):
    quotation.append(quote[qno].text.strip())
quotation

In [ ]:
a = soup.select('.tborder.vbseo_like_postbit')[6].find("div", id=re.compile("^post_message_"))
quote = soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')
quotation = []

if len(quote) > 0:
    for quo_no in range(0, len(quote)):
        a.select('td.alt2')[0].extract()
    print(a.text)
else:
    print(a.text)

In [ ]:
a = soup.select('.tborder.vbseo_like_postbit')[6].find("div", id=re.compile("^post_message_"))
quote = soup.select('.tborder.vbseo_like_postbit')[6].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')
quotation = []

if len(quote) > 0:
    for qno in range(0,len(quote)):
        quotation.append(quote[qno].text.strip())
    for quo_no in range(0, len(quote)):
        a.select('td.alt2')[0].extract()
    print(a.text)
else:
    print(a.text)

In [ ]:
quotation

## For method 1: Handle with content, etract div & quotation

In [ ]:
title_url = 'http://www.dogforum.com/dog-health/what-parvo-5256/page2/'
resp = requests.get(title_url, headers=random_header(title_url))
resp.encoding = 'cp1252'
soup = BeautifulSoup(resp.text, 'lxml')

In [ ]:
a = soup.select('.tborder.vbseo_like_postbit')[8].find("div", id=re.compile("^post_message_"))
quote = soup.select('.tborder.vbseo_like_postbit')[8].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')

if len(quote) > 0:
    quotation = []
    for qno in range(0,len(quote)):
        quotation.append(quote[qno].text.strip())
    block['quotation'] = quotation
    for quo_no in range(0, len(quote)):
        a.select('td.alt2')[0].extract()
        if len(a.select('div')) > 0:
            a.select('div')[0].extract()
else:
    if len(a.select('div')) > 0:
        a.select('div')[0].extract()
block['content'] = a.text.strip().replace('Quote:','[...quotation...]').replace("\r","").replace("\n","").replace('\t','')
print(block['content'])

In [ ]:
quotation

## For method 1: Combine Together

In [ ]:
# Method 1 (only remove div one times)
title_url = 'http://www.dogforum.com/dog-health/what-parvo-5256/page2/'
resp = requests.get(title_url, headers=random_header(title_url))
resp.encoding = 'cp1252'
soup = BeautifulSoup(resp.text, 'lxml')

block = {}

for num in range(0,len(soup.select('.tborder.vbseo_like_postbit'))):
    post_date = {}
    quotation = []
    
    block['url'] = title_url
    block['page'] = 'page %s'%page
    block['title'] = title
    block['story'] = soup.select('.tborder.vbseo_like_postbit')[num].findAll(id=re.compile("^postcount"))[0].get('name')
    block['author_url'] = soup.select('.tborder.vbseo_like_postbit')[num].select('a.bigusername')[0]['href']
    block['author_name'] = soup.select('.tborder.vbseo_like_postbit')[num].select('a.bigusername')[0].text

    data = {}
    data['date'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[0].replace(',','')
    data['time'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[1]
    data['ampm'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[2]
    block['post_date'] = data

    author = {}
    try:
        author['member_type'] = soup.select('.tborder.vbseo_like_postbit')[num].select('div.smallfont')[0].text
    except IndexError:
        pass
    try:
        author['location'] = re.findall('Location: (.*).', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
    except IndexError:
        pass
    try:
        author['join_date'] = re.findall('Join Date:(.*) Location', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
    except IndexError:
        pass
    try:
        author['posts'] = re.findall('Posts: (.*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
    except IndexError:
        pass
    try:
        author['mentioned'] = re.findall('Mentioned: (.*)Tag', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
    except IndexError:
        pass
    try:
        author['tagged'] = re.findall('Tagged: (\d*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip() + ' thread(s)'
    except IndexError:
        pass
    block['author_info'] = author

    a = soup.select('.tborder.vbseo_like_postbit')[num].find("div", id=re.compile("^post_message_"))
    quote = soup.select('.tborder.vbseo_like_postbit')[num].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')
    if len(quote) > 0:
        for qno in range(0,len(quote)):
            quotation.append(quote[qno].text.strip())
        for quo_no in range(0, len(quote)):
            a.select('td.alt2')[0].extract()
            if len(a.select('div')) > 0:
                a.select('div')[0].extract()
    else:
        if len(a.select('div')) > 0:
            a.select('div')[0].extract()
    block['content'] = a.text.strip().replace('Quote:','[...quotation...]').replace("\r","").replace("\n","").replace('\t','')
    block['quotation'] = quotation
    print(block)

In [64]:
def to_mongoDB(section):
    conn = MongoClient('localhost', 27017)
    db = conn.Dogforum
    collection = db.Dog_health
    results = collection.insert_many(section)
    conn.close()

In [65]:
def to_rethinkdb(section):
    conn = r.connect()
    res = r.db('Dogforum').table('test').insert(section).run(conn)
    conn.close()

In [14]:
# Method 2 (remove div without content)
title_url = 'http://www.dogforum.com/dog-health/what-parvo-5256/page2/'
resp = requests.get(title_url, headers=random_header(title_url))
resp.encoding = 'utf-8'
soup = BeautifulSoup(resp.text, 'lxml')
page = 2
title = 'test'

post = []

for num in range(0,len(soup.select('.tborder.vbseo_like_postbit'))):
    block = {}
    post_date = {}
    quotation = []
    
    block['url'] = title_url
    block['page'] = 'page %s'%page
    block['title'] = title
    floor = soup.select('.tborder.vbseo_like_postbit')[num].findAll(id=re.compile("^postcount"))[0].get('name')
    block['floor'] = int(floor)
    block['author_url'] = soup.select('.tborder.vbseo_like_postbit')[num].select('a.bigusername')[0]['href']
    block['author_name'] = soup.select('.tborder.vbseo_like_postbit')[num].select('a.bigusername')[0].text

    data = {}
    data['date'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[0].replace(',','')
    data['time'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[1]
    data['ampm'] = soup.select('.tborder.vbseo_like_postbit')[num].find('td').text.split()[2]
    block['post_date'] = data

    author = {}
    try:
        author['member_type'] = soup.select('.tborder.vbseo_like_postbit')[num].select('div.smallfont')[0].text
    except IndexError:
        pass
    try:
        author['location'] = re.findall('Location: (.*).', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
    except IndexError:
        pass
    try:
        author['join_date'] = re.findall('Join Date:(.*) Location', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
    except IndexError:
        pass
    try:
        author['posts'] = re.findall('Posts: (.*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
    except IndexError:
        pass
    try:
        author['mentioned'] = re.findall('Mentioned: (.*)Tag', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip()
    except IndexError:
        pass
    try:
        author['tagged'] = re.findall('Tagged: (\d*)', soup.select('.tborder.vbseo_like_postbit')[num].text)[0].strip() + ' thread(s)'
    except IndexError:
        pass
    block['author_info'] = author
    block['_id'] = 'Page%s'%page + 'Floor%s'%floor + re.findall('http://www.dogforum.com(.*)', title_url)[0]
    block['id'] = 'Page%s'%page + 'Floor%s'%floor + re.findall('http://www.dogforum.com(.*)', title_url)[0]

    quotation = [y.text.strip() for y in soup.select('.tborder.vbseo_like_postbit')[num].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')]
    [div.extract() for div in soup.select('.tborder.vbseo_like_postbit')[num].find("div", id=re.compile("^post_message_")).select('div')]
    [quo.extract() for quo in soup.select('.tborder.vbseo_like_postbit')[num].findAll(id=re.compile("^post_message_"))[0].select('td.alt2')]
    a = soup.select('.tborder.vbseo_like_postbit')[num].find("div", id=re.compile("^post_message_")).text
    a.replace('Sent from my iPad using Tapatalk','').replace('Quote: ','[...quotation...]').replace("\r","").replace('\t','').strip()
    block['content'] = a
    block['quotation'] = quotation
    post.append(block)
post
#to_rethinkdb(post)
#to_mongoDB(post)

[{'_id': 'Page2Floor11/dog-health/what-parvo-5256/page2/',
  'author_info': {'member_type': 'Senior Member',
   'mentioned': '0 Post(s)',
   'posts': '12,135',
   'tagged': '0 thread(s)'},
  'author_name': 'pawzaddict',
  'author_url': 'http://www.dogforum.com/members/737/',
  'content': "  Its different per dog. Some dogs have actually never showed severe symptoms but that's rare Posted via Mobile Device ",
  'floor': 11,
  'id': 'Page2Floor11/dog-health/what-parvo-5256/page2/',
  'page': 'page 2',
  'post_date': {'ampm': 'PM', 'date': '08-10-2010', 'time': '04:12'},
  'quotation': [],
  'title': 'test',
  'url': 'http://www.dogforum.com/dog-health/what-parvo-5256/page2/'},
 {'_id': 'Page2Floor12/dog-health/what-parvo-5256/page2/',
  'author_info': {'join_date': 'Jun 2010',
   'location': 'At a desk, in a chair, in front of a monitor and keyboard',
   'member_type': 'Senior Member',
   'mentioned': '0 Post(s)',
   'posts': '152',
   'tagged': '0 thread(s)'},
  'author_name': 'MorningG